Analysis Results

In [1]:
# Connect Google drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# change directory to mistaken
!mkdir -p drive
!google-drive-ocamlfuse drive

In [1]:
!ls

datalab  drive


In [2]:
import os
os.chdir('drive')
os.chdir('mistaken')
!ls -l 

total 136475072
drwxr-xr-x 2 root root        4096 Mar  5 06:24 afs
drwxr-xr-x 2 root root        4096 Apr  4 04:26 afs1
-rw-r--r-- 1 root root   353725242 Apr  4 04:22 afs1.zip
-rw-r--r-- 1 root root       79141 Aug  1 15:39 analysis_results.ipynb
-rw-r--r-- 1 root root        1569 Jun 20 14:31 a.npy
drwxr-xr-x 2 root root        4096 Jun 14 05:47 batch_files
-rw-r--r-- 1 root root       34151 Mar  5 09:27 big_living_background.png
-rw-r--r-- 1 root root       47809 Mar  5 09:27 big_park_background.png
-rw-r--r-- 1 root root     7286409 Mar  5 06:26 caffe
-rw-r--r-- 1 root root        9321 May 12 05:55 concatenate_feature.py
drwxr-xr-x 2 root root        4096 Apr 11 11:00 concatenate_feature_resnet50
-rw-r--r-- 1 root root      581355 May  7 04:00 concat_indices_200000000000.json
-rw-r--r-- 1 root root      586640 Apr 16 12:36 concat_indices_2223320934302811922.json
-rw-r--r-- 1 root root      586640 Jun 12 08:10 concat_indices_2408.json
-rw-r--r-- 1 root root      586640 Apr  9 06:12

In [4]:
# set feature_hash and experiment_hash
# and the folder contains y_hat and y_prob file

result_folder = os.path.join('results', 'alexnet_7_1___2')
print("Result folder: ", result_folder)

('Result folder: ', 'results/alexnet_7_1___2')


In [3]:
# import library
!pip install tqdm
import matplotlib
matplotlib.use('Agg')

import argparse
import copy
import glob
import h5py
import json
import matplotlib.pyplot as plt
from multiprocessing import Pool
import numpy as np
import os
from render_scenes_json import RenderScenes
from scipy.misc import imsave, imresize
from scipy.ndimage import imread
import shutil
from sklearn.metrics import confusion_matrix, average_precision_score, f1_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import sys
from tqdm import tqdm
import util

#sys.path.append("/home/ngocthach/caffe/python")

########################################################################
# Global Variables

DATA_PATH = 'data'                              # Where to store temporary files
PUBLIC_PATH = 'public'                          # Where to show graphs and diagnostics
EGOCENTRIC_IMAGE_FOLDER = 'egocentric_images'   # Where egocentric images are stored
TMP_PATH = 'tmp'  


MISTAKEN_TASK                       = True
WHO_TASK                            = True
WHEN_TASK                           = True

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "<decorator-gen-121>", line 2, in initialize
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 87, in catch_

In [14]:
# set feature_hash and experiment_hash
# and the folder contains y_hat and y_prob file

result_folder = os.path.join('results', 'alexnet_singlelayer')
print("Result folder: ", result_folder)

('Result folder: ', 'results/alexnet_singlelayer')


In [0]:
random_seed_balanced = 0

In [16]:
#@title Default title text
#### Get train/test results ########################

print("Loading file")
#feature_hash = 2408
feature_hash = 438455043325458450
#feature_hash = 2408
experiment_hash_list = [4754952150018512305, 8647626824687750802, 8949258729311709173, 1182471099183327604]
experiment_hash_list = [1182471099183327604]
#experiment_hash_list = [8841606304273805265, 1516386097573094802, 5305626107254735877, 2983505177517327716]
RANDOM_SEED = -1


for experiment_hash in experiment_hash_list:

    ####################################################################################

    # Mistaken task:
    # 1. Given a character and a frame, is the character mistaken in this frame?
    # 2. Given a character and a scene, is the character mistaken in any frame in the scene?
    
    print("############################################################")
    RANDOM_SEED += 1
    print("Random_seed: ", RANDOM_SEED)
    print("Feature_hash: ", feature_hash)
    print("Experiment_hash: ", experiment_hash)
    
    if MISTAKEN_TASK:
        print 'Mistaken task'
        y = np.load('y_%d.npy' % feature_hash)
        y_prob = np.load(os.path.join(result_folder, 'y_prob_%d.npy' % experiment_hash))
        t = np.load('t_%d.npy' % feature_hash)
        y_hat = np.load(os.path.join(result_folder, 'y_hat_%d.npy' % experiment_hash))
        split_mask = np.load('split_mask_%d.npy' % experiment_hash)

        y_prob_test = y_prob[split_mask == 'test']
        y_train = y[split_mask == 'train']
        y_val = y[split_mask == 'val']
        y_test = y[split_mask == 'test']

        y_train_hat = y_hat[split_mask == 'train']
        y_val_hat = y_hat[split_mask == 'val']
        y_test_hat = y_hat[split_mask == 'test']

        t_train = t[split_mask == 'train']
        t_val = t[split_mask == 'val']
        t_test = t[split_mask == 'test']
        y_test_mistaken_frame_unbalanced = y_test

        print 'Mistaken Task - single frame'

        print 'Training Accuracy: %.3f%%' % (100.0 * util.accuracy_score(y_train, y_train_hat))
        print 'Confusion matrix:'
        print confusion_matrix(y_train, y_train_hat)
        print 'Accuracy vs time:', [100.0 * util.accuracy_score(y_train[t_train == i], y_train_hat[t_train == i]) for i in range(8)]
        print '\n'
        print 'Validation Accuracy: %.3f%%' % (100.0 * util.accuracy_score(y_val, y_val_hat))
        print 'Confusion matrix:'
        print confusion_matrix(y_val, y_val_hat)
        print 'Accuracy vs time:', [100.0 * util.accuracy_score(y_val[t_val == i], y_val_hat[t_val == i]) for i in range(8)]
        print '\n'

        print 'Test Accuracy: %.3f%%' % (100.0 * util.accuracy_score(y_test, y_test_hat))
        #print 'Test F-measure:  %.3f' %f1_score(y_test, y_test_hat)
        print 'Confusion matrix:'
        print confusion_matrix(y_test, y_test_hat)
        print 'Accuracy vs time:', [100.0 * util.accuracy_score(y_test[t_test == i], y_test_hat[t_test == i]) for i in range(8)]
        print '\n'
        ((y_test_hat, y_prob_test), y_test) = util.balance((y_test_hat, y_prob_test), y_test, random_seed=random_seed_balanced)
        y_test_mistaken_frame_balanced = y_test
        print 'Mistaken[frame]: Accuracy: %.3f%%' % (100.0 * util.accuracy_score(y_test, y_test_hat))
        #print 'Mistaken[frame] F-measure:  %.3f' %f1_score(y_test, y_test_hat)
        print 'Mistaken[frame]: AP: %.3f' % average_precision_score(y_test, y_prob_test)
        print confusion_matrix(y_test, y_test_hat)
        print 'Mistaken[frame]: Chance: %.3f%%' % (100.0 * np.mean(y_test == 1))
        print 'Mistaken[frame]: Chance AP: %.3f' % average_precision_score(y_test, np.mean(y_test) * np.ones_like(y_test))
        print

        # Mistaken[scene] - given a character and a scene, is the character mistaken in any frame in the scene?

        question_dict_filename = os.path.join(DATA_PATH, 'question_dict.json')

        split_dict_filename = os.path.join(DATA_PATH, 'split_dict_%02d.json' % RANDOM_SEED)


        with open(split_dict_filename) as f:
            split_dict = json.load(f)

        with open(question_dict_filename) as f:
            question_dict = json.load(f)

        with open('concat_indices_%d.json' % feature_hash, 'rb') as f:
            concat_indices = json.load(f)


        val_min_predict = []
        val_avg_predict = []
        val_y = []
        test_min_predict = []
        test_avg_predict = []
        test_y = []

        for frame_key in tqdm(question_dict):
            (scene_id, frame_index) = frame_key.split('_')
            frame_index = int(frame_index)

            # We only want to loop through each scene once
            if frame_index != 0:
                continue

            # We only want to evaluate on characters who were mistaken in every frame or correct in every frame
            # In the loop below, the answers for characters who are not in the frame may be incorrect.
            # This is OK, because in the second loop, we only look at frames whether the model made predictions (i.e. the character was present)

            confused_dict = dict()

            for frame_index in range(8):
                frame_key = '%s_%d' % (scene_id, frame_index)
                answer_dict_vec = question_dict[frame_key]['answers']
                character_vec = question_dict[frame_key]['characters']
                for character_name in character_vec:
                    key = '%s_%s_%s' % (scene_id, frame_index, character_name)
                    if 'Doll' in character_name and key in concat_indices:
                        answer_vec = []
                        for answer_dict in answer_dict_vec:
                            answer = int(answer_dict[character_name] == answer_dict['omniscient'] )
                            answer_vec.append(answer)

                        if answer_vec[0] == answer_vec[1]:
                            confused_dict[character_name] = [answer] + confused_dict.get(character_name, [])


            for (character_name, answer_vec) in confused_dict.items():
                correct = min(answer_vec)

                predict = []
                for frame_index in range(8):
                    key = '%s_%s_%s' % (scene_id, frame_index, character_name)
                    # A character may appear in only a subset of the 8 frames. We
                    # predict whether he is mistaken based on predictions from those
                    # frames.
                    if key in concat_indices:
                        index = concat_indices[key]
                        predict.append(y_prob[index])

                assert len(predict) > 0
                min_predict = min(predict)
                avg_predict = np.mean(predict)

                if split_dict[scene_id] == 'val':
                    val_min_predict.append(min_predict)
                    val_avg_predict.append(avg_predict)
                    val_y.append(correct)

                if split_dict[scene_id] == 'test':
                    test_min_predict.append(min_predict)
                    test_avg_predict.append(avg_predict)
                    test_y.append(correct)

        test_y = np.array(test_y)
        test_min_predict = np.array(test_min_predict)
        test_avg_predict = np.array(test_avg_predict)
        y_test_mistaken_scene_unbalanced = test_y

        ((test_min_predict, test_avg_predict), test_y) = util.balance((test_min_predict, test_avg_predict), test_y, random_seed=random_seed_balanced)
        y_test_mistaken_scene_balanced = test_y

        print confusion_matrix(test_y, test_min_predict>0.5)
        print 'Mistaken[scene]: Accuracy (min pool): %.3f%%' % (100.0 * util.accuracy_score(test_y, test_min_predict > 0.5))
        #print 'Mistaken[scene] F-measure (min pool):  %.3f'% f1_score(test_y, test_min_predict>0.5)
        print 'Mistaken[scene]: AP (min pool): %.3f' % average_precision_score(test_y, test_min_predict)
        print 'Mistaken[scene]: Accuracy (avg pool): %.3f%%' % (100.0 * util.accuracy_score(test_y, test_avg_predict > 0.5))
        #print 'Mistaken[scene] F-measure (avg pool):  %.3f'% f1_score(test_y, test_avg_predict>0.5)
        print 'Mistaken[scene]: AP (avg pool): %.3f' % average_precision_score(test_y, test_avg_predict)
        print confusion_matrix(test_y, test_avg_predict>0.5)
        print 'Mistaken[scene]: Chance: %.3f%%' % (100.0 * max(np.mean(test_y), 1 - np.mean(test_y)))
        print 'Mistaken[scene]: Chance AP: %.3f' % average_precision_score(test_y, np.mean(test_y) * np.ones_like(test_y))
        print



    ####################################################################################


    if WHO_TASK:
        question_dict_filename = os.path.join(DATA_PATH, 'question_dict.json')

        split_dict_filename = os.path.join(DATA_PATH, 'split_dict_%02d.json' % RANDOM_SEED)

        y_prob = np.load(os.path.join(result_folder, 'y_prob_%d.npy' % experiment_hash))

        with open(split_dict_filename) as f:
            split_dict = json.load(f)

        with open(question_dict_filename) as f:
            question_dict = json.load(f)

        with open('concat_indices_%d.json' % feature_hash, 'rb') as f:
            concat_indices = json.load(f)

        # Who task for a single frame:
        # Given a frame with a mistaken character and a non-mistaken character, predict which character is mistaken
        # Note: If a frame contains more than two characters, we will compute this for all pairs of (mistaken, non-mistaken) characters in this frame

        correct = 0
        incorrect = 0
        predict_vec = []
        y_vec = []
        for frame_key in tqdm(question_dict):
            (scene_id, frame_index) = frame_key.split('_')
            if split_dict[scene_id] == 'test':
                answer_dict_vec = question_dict[frame_key]['answers']
                answer_vec = []
                confused_dict = dict()
                for answer_dict in answer_dict_vec:
                    for character_name in answer_dict:
                        if 'Doll' in character_name:
                            answer = int(answer_dict[character_name] == answer_dict['omniscient'])
                            confused_dict[character_name] = [answer] + confused_dict.get(character_name, [])
                consistent_confused_dict = dict()
                for (character_name, answers) in confused_dict.items():
                    if answers[0] == answers[1]:
                        consistent_confused_dict[character_name] = answers[0]
                for (character_name_1, correct_1) in consistent_confused_dict.items():
                    for (character_name_2, correct_2) in consistent_confused_dict.items():
                        if correct_1 == 0 and correct_2 == 1:
                            assert character_name_1 != character_name_2
                            key_1 = '%s_%s_%s' % (scene_id, frame_index, character_name_1)
                            key_2 = '%s_%s_%s' % (scene_id, frame_index, character_name_2)
                            if key_1 in concat_indices and key_2 in concat_indices:
                                index_1 = concat_indices[key_1]
                                index_2 = concat_indices[key_2]
                                if y_prob[index_1] < y_prob[index_2]:
                                    correct += 1
                                elif y_prob[index_1] == y_prob[index_2]:
                                    correct += 0.5
                                    incorrect += 0.5
                                else:
                                    incorrect += 1

        print 'Who[frame]: Accuracy: %.3f%%' % (100.0 * float(correct) / (correct + incorrect))

        # Who task for a scene:
        # Given a scene with a mistaken character and a non-mistaken character, predict which character is mistaken
        # Note: If a scene contains more than two characters, we will compute this for all pairs of (mistaken, non-mistaken) characters in this scene

        # We can aggregate predictions across frames by taking the mean or min
        min_correct = 0
        min_incorrect = 0
        mean_correct = 0
        mean_incorrect = 0
        for frame_key in tqdm(question_dict):
            (scene_id, frame_index) = frame_key.split('_')
            frame_index = int(frame_index)

            # We only want to loop through each scene once
            if frame_index != 0:
                continue

            if split_dict[scene_id] != 'test':
                continue

            # Create confused dict, which says which characters are consistently confused
            confused_dict = dict()
            for frame_index in range(8):
                frame_key = '%s_%d' % (scene_id, frame_index)
                answer_dict_vec = question_dict[frame_key]['answers']
                character_vec = question_dict[frame_key]['characters']
                for character_name in character_vec:
                    key = '%s_%s_%s' % (scene_id, frame_index, character_name)
                    answer_vec = []
                    if 'Doll' in character_name and key in concat_indices:
                        for answer_dict in answer_dict_vec:
                            answer = int(answer_dict[character_name] == answer_dict['omniscient'])
                            answer_vec.append(answer)
                        if answer_vec[0] == answer_vec[1]:
                            confused_dict[character_name] = [answer] + confused_dict.get(character_name, [])

            for (character_name_1, answers_1) in confused_dict.items():
                for (character_name_2, answers_2) in confused_dict.items():
                    correct_1 = min(answers_1)
                    correct_2  = min(answers_2)

                    if correct_1 == 0 and correct_2 == 1:
                        predict_1 = []
                        predict_2 = []
                        for frame_index in range(8):
                            key_1 = '%s_%s_%s' % (scene_id, frame_index, character_name_1)
                            key_2 = '%s_%s_%s' % (scene_id, frame_index, character_name_2)
                            if key_1 in concat_indices:
                                index_1 = concat_indices[key_1]
                                predict_1.append(y_prob[index_1])

                            if key_2 in concat_indices:
                                index_2 = concat_indices[key_2]
                                predict_2.append(y_prob[index_2])

                        assert len(predict_1) > 0 and len(predict_2) > 0
                        if np.min(predict_1) < np.min(predict_2):
                            min_correct += 1.0
                        elif np.min(predict_1) == np.min(predict_2):
                            min_correct += 0.5
                            min_incorrect += 0.5
                        else:
                            min_incorrect += 1.0

                        if np.mean(predict_1) < np.mean(predict_2):
                            mean_correct += 1.0
                        elif np.mean(predict_1) == np.mean(predict_2):
                            mean_correct += 0.5
                            mean_incorrect += 0.5
                        else:
                            mean_incorrect += 1.0

        print 'Who[scene]: Accuracy (min pool): %.3f%%' % (100.0 * float(min_correct) / (min_correct + min_incorrect))
        print 'Who[scene]: Accuracy (avg pool): %.3f%%' % (100.0 * float(mean_correct) / (mean_correct + mean_incorrect))


    if WHEN_TASK:
        count_ambiguous = 0
        # Given a frame, is any character mistaken in the frame?
        question_dict_filename = os.path.join(DATA_PATH, 'question_dict.json')

        split_dict_filename = os.path.join(DATA_PATH, 'split_dict_%02d.json' % RANDOM_SEED)
        y_prob = np.load(os.path.join(result_folder, 'y_prob_%d.npy' % experiment_hash))

        with open(split_dict_filename) as f:
            split_dict = json.load(f)

        with open(question_dict_filename) as f:
            question_dict = json.load(f)

        with open('concat_indices_%d.json' % feature_hash, 'rb') as f:
            concat_indices = json.load(f)

        min_predict_vec = []
        avg_predict_vec = []
        correct_vec = []
        for frame_key in question_dict:
            (scene_id, frame_index) = frame_key.split('_')
            frame_index = int(frame_index)

            if split_dict[scene_id] != 'test':
                continue

            # Create confused dict, which says which characters are consistently confused
            confused_dict = dict()

            answer_dict_vec = question_dict[frame_key]['answers']
            for answer_dict in answer_dict_vec:
                for character_name in answer_dict:
                    key = '%s_%s_%s' % (scene_id, frame_index, character_name)
                    if 'Doll' in character_name and key in concat_indices:
                        answer = int(answer_dict[character_name] == answer_dict['omniscient'])
                        confused_dict[character_name] = [answer] + confused_dict.get(character_name, [])

            consistent_frame = True
            for (character_name, answers) in confused_dict.items():
                if len(set(answers)) > 1:
                    #consistent_frame = False
                    #break
                    count_ambiguous += 1
                    del confused_dict[character_name]

            if not consistent_frame or len(confused_dict.keys()) == 0:
                continue

            # answer tells us whether every character is correct.
            # if the frame contains a consistently mistaken character (across the two annotations), answer = 0
            answer = min([ans for ans_vec in confused_dict.values() for ans in ans_vec])
            predict = []

            for character_name in confused_dict.keys():
                key = '%s_%s_%s' % (scene_id, frame_index, character_name)
                if key in concat_indices:
                    index = concat_indices[key]
                    predict.append(y_prob[index])

            min_predict = min(predict)
            avg_predict = np.mean(predict)


            min_predict_vec.append(min_predict)
            avg_predict_vec.append(avg_predict)
            correct_vec.append(answer)


        min_predict_vec = np.array(min_predict_vec)
        avg_predict_vec = np.array(avg_predict_vec)
        correct_vec = np.array(correct_vec)
        y_test_when_unbalanced = correct_vec

        ((min_predict_vec, avg_predict_vec), correct_vec) = util.balance((min_predict_vec, avg_predict_vec), correct_vec, random_seed=random_seed_balanced)
        y_test_when_balanced = correct_vec
        print("Ambiguous: ", count_ambiguous)
        print confusion_matrix(correct_vec, min_predict_vec >0.5)
        print 'When: Accuracy (min pool): %.3f%%' % (100.0 * util.accuracy_score(correct_vec, min_predict_vec > 0.5))
        #print 'Mistaken[scene] F-measure (min pool):  %.3f' %f1_score(correct_vec, min_predict_vec >0.5)
        print 'When: AP (min pool): %.3f' % average_precision_score(correct_vec, min_predict_vec)
        print 'When: Accuracy (avg pool): %.3f%%' % (100.0 * util.accuracy_score(correct_vec, avg_predict_vec > 0.5))
        print 'When: AP (avg pool): %.3f' % average_precision_score(correct_vec, avg_predict_vec)
        print confusion_matrix(correct_vec, avg_predict_vec >0.5)
        #print 'When F-measure (avg pool):  %.3f'% f1_score(correct_vec, avg_predict_vec >0.5)
        print 'When: Chance: %.3f%%' % (100.0 * np.mean(correct_vec))
        print 'When: Chance AP: %.3f' % average_precision_score(correct_vec, np.mean(correct_vec) * np.ones_like(correct_vec))


    ########################################################################
    # Cleanup

print 'Cleaning up'

Loading file
############################################################
('Random_seed: ', 0)
('Feature_hash: ', 438455043325458450)
('Experiment_hash: ', 1182471099183327604)
Mistaken task
Mistaken Task - single frame
Training Accuracy: 70.983%
Confusion matrix:
[[1788  443]
 [2356 5059]]
Accuracy vs time: [84.86486486486487, 85.20710059171599, 77.53378378378379, 60.1063829787234, 60.2224123182207, 54.6462063086104, 56.91244239631337, 86.97208303507516]


Validation Accuracy: 65.802%
Confusion matrix:
[[216  82]
 [287 494]]
Accuracy vs time: [79.82456140350878, 80.0, 70.22900763358778, 62.22222222222222, 51.14503816793893, 51.449275362318836, 54.861111111111114, 78.2051282051282]


Test Accuracy: 66.098%
Confusion matrix:
[[202  96]
 [321 611]]
Accuracy vs time: [84.02777777777779, 77.27272727272727, 66.66666666666666, 53.284671532846716, 47.682119205298015, 51.92307692307693, 59.523809523809526, 85.79545454545455]


Mistaken[frame]: Accuracy: 66.416%
Mistaken[frame]: AP: 0.683
[[627

100%|██████████| 9704/9704 [00:00<00:00, 36477.80it/s]


[[109   8]
 [ 73  44]]
Mistaken[scene]: Accuracy (min pool): 65.385%
Mistaken[scene]: AP (min pool): 0.805
Mistaken[scene]: Accuracy (avg pool): 72.650%
Mistaken[scene]: AP (avg pool): 0.762
[[79 38]
 [26 91]]
Mistaken[scene]: Chance: 50.000%
Mistaken[scene]: Chance AP: 0.500



100%|██████████| 9704/9704 [00:00<00:00, 158313.79it/s]


Who[frame]: Accuracy: 87.770%
Who[scene]: Accuracy (min pool): 80.000%
Who[scene]: Accuracy (avg pool): 80.000%
('Ambiguous: ', 0)
[[464  81]
 [205 340]]
When: Accuracy (min pool): 73.761%
When: AP (min pool): 0.841
When: Accuracy (avg pool): 72.661%
When: AP (avg pool): 0.807
[[409 136]
 [162 383]]
When: Chance: 50.000%
When: Chance AP: 0.500
Cleaning up


Load model

In [5]:
# set feature_hash and experiment_hash
# and the folder contains y_hat and y_prob file

experiment_hash = 4754952150018512305

result_folder = os.path.join('results', 'alexnet_14_7_1')
print("Result folder: ", result_folder)

('Result folder: ', 'results/alexnet_14_7_1')


In [0]:
import pickle

In [7]:
#experiment_hash_list = [4754952150018512305, 8647626824687750802, 8949258729311709173, 1182471099183327604]
experiment_hash_list = [8647626824687750802, 8949258729311709173, 1182471099183327604]
for experiment_hash in experiment_hash_list:
  with open(os.path.join(result_folder, 'training_history_%d'%experiment_hash), mode='rb') as file_history:
    history = pickle.load(file_history)

    print(history.keys())
    # summarize history for accuracy
    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

['acc', 'loss', 'val_acc', 'val_loss']


NameError: ignored

In [20]:
sum(test_avg_predict > 0.5)/float(len(test_avg_predict))

0.5512820512820513

Evaluate train/test

In [0]:
def evaluate_test(y_test):
  print("Len: ", len(y_test))
  print("#mistaken: ", sum(y_test==0))
  print("Mistaken %: ", sum(y_test==0)/float(len(y_test)))

In [0]:
y_test_dict = {
    'mistaken_frame_unbalanced': y_test_mistaken_frame_unbalanced,
    'mistaken_frame_balanced': y_test_mistaken_frame_balanced,
    'mistaken_scene_unbalanced': y_test_mistaken_scene_unbalanced,
    'mistaken_scene_balanced': y_test_mistaken_scene_balanced,
    'when_unbalacned': y_test_when_unbalanced, 
    'when_balanced': y_test_when_balanced
}

In [23]:
for (key, value) in y_test_dict.items():
  print(key)
  evaluate_test(value)

mistaken_scene_balanced
('Len: ', 234)
('#mistaken: ', 117)
('Mistaken %: ', 0.5)
mistaken_scene_unbalanced
('Len: ', 232)
('#mistaken: ', 117)
('Mistaken %: ', 0.5043103448275862)
mistaken_frame_unbalanced
('Len: ', 1230)
('#mistaken: ', 298)
('Mistaken %: ', 0.24227642276422764)
when_balanced
('Len: ', 1090)
('#mistaken: ', 545)
('Mistaken %: ', 0.5)
mistaken_frame_balanced
('Len: ', 1864)
('#mistaken: ', 932)
('Mistaken %: ', 0.5)
when_unbalacned
('Len: ', 759)
('#mistaken: ', 214)
('Mistaken %: ', 0.28194993412384717)


In [0]:
split_dict_filename = os.path.join(DATA_PATH, 'split_dict_00.json')
with open(split_dict_filename) as f:
    split_dict = json.load(f)


In [25]:
splitA = {'train':0, 'test':0, 'val':0, 'other':0}
for (key, value) in split_dict.items():
  if value in ['train', 'val', 'test']:
    splitA[value] += 1
  else:
    splitA['other'] += 1

print(splitA)

{'test': 122, 'train': 970, 'other': 0, 'val': 121}


Evaluate the prediction of muliple models

In [4]:
feature_hash = 438455043325458450
experiment_hash = 8647626824687750802

# set feature_hash and experiment_hash
# and the folder contains y_hat and y_prob file

result_folder = 'results'
print("Result folder: ", result_folder)

('Result folder: ', 'results')


In [0]:
list_model = ['alexnet_singlelayer', 'alexnet_4_1', 'alexnet_7_1', 'alexnet_11_1', 'alexnet_15_1', 'alexnet_7_3_1', 'alexnet_14_7_1']

# read y test
y = np.load('y_%d.npy' % feature_hash)

y_dict = {'correct_label': y}


# read y_hat and y_prob of each model

for model in list_model:
  result_folder_temp = os.path.join(result_folder, model)
  y_prob = np.load(os.path.join(result_folder_temp,'y_prob_%d.npy' % experiment_hash))
  y_hat = np.load(os.path.join(result_folder_temp,'y_hat_%d.npy' % experiment_hash))
  
  y_dict[model] = {'y_hat':y_hat, 'y_prob': y_prob}

In [0]:

split_mask = np.load('split_mask_%d.npy' % experiment_hash)
prediction_dict = {}
for model in list_model:
  prediction_dict[model] = {'correct_mistaken': [], 'wrong_mistaken':[], 'correct_non_mistaken': [], 'wrong_non_mistaken': []}

# read concat_indices file
with open('concat_indices_%d.json' % feature_hash, 'rb') as f:
    concat_indices = json.load(f)
    
for (scene_frame_character, index) in concat_indices.items():
  
  # check if index is for test set
  if split_mask[index] != 'test':
    continue
  
  for model in list_model:
    if y_dict['correct_label'][index] == 0:
      
      if y_dict['correct_label'][index] == y_dict[model]['y_hat'][index]:
        prediction_dict[model]['correct_mistaken'].append(scene_frame_character)
      else:
        prediction_dict[model]['wrong_mistaken'].append(scene_frame_character)
        
    elif y_dict['correct_label'][index] == 1:
      
      if y_dict['correct_label'][index] == y_dict[model]['y_hat'][index]:
        prediction_dict[model]['correct_non_mistaken'].append(scene_frame_character)
      else:
        prediction_dict[model]['wrong_non_mistaken'].append(scene_frame_character)
      
      

In [7]:
for model in list_model:
  print("Model: ", model)
  print("#Wrong mistaken: ", len(prediction_dict[model]['wrong_mistaken']))
  print("#Correct mistaken: ", len(prediction_dict[model]['correct_mistaken']))
  print("#Wrong non mistaken: ", len(prediction_dict[model]['wrong_non_mistaken']))
  print("#Correct non mistaken: ", len(prediction_dict[model]['correct_non_mistaken']))

('Model: ', 'alexnet_singlelayer')
('#Wrong mistaken: ', 98)
('#Correct mistaken: ', 193)
('#Wrong non mistaken: ', 302)
('#Correct non mistaken: ', 557)
('Model: ', 'alexnet_4_1')
('#Wrong mistaken: ', 122)
('#Correct mistaken: ', 169)
('#Wrong non mistaken: ', 201)
('#Correct non mistaken: ', 658)
('Model: ', 'alexnet_7_1')
('#Wrong mistaken: ', 80)
('#Correct mistaken: ', 211)
('#Wrong non mistaken: ', 305)
('#Correct non mistaken: ', 554)
('Model: ', 'alexnet_11_1')
('#Wrong mistaken: ', 137)
('#Correct mistaken: ', 154)
('#Wrong non mistaken: ', 168)
('#Correct non mistaken: ', 691)
('Model: ', 'alexnet_15_1')
('#Wrong mistaken: ', 128)
('#Correct mistaken: ', 163)
('#Wrong non mistaken: ', 179)
('#Correct non mistaken: ', 680)
('Model: ', 'alexnet_7_3_1')
('#Wrong mistaken: ', 156)
('#Correct mistaken: ', 135)
('#Wrong non mistaken: ', 127)
('#Correct non mistaken: ', 732)
('Model: ', 'alexnet_14_7_1')
('#Wrong mistaken: ', 3)
('#Correct mistaken: ', 288)
('#Wrong non mistaken: '

In [8]:
# compare prediction class of baseline vs model 7_1 


comparison_dict = {'all_correct': {}, 'all_wrong': {}, 'alexnet_singlelayer_correct': {}, model: {}}
for model in list_model:
  
  if model == 'alexnet_singlelayer':
    continue


  comparison_dict['all_correct'] = {'mistaken': [], 'non_mistaken':[]}
  comparison_dict['all_wrong'] = {'mistaken': [], 'non_mistaken':[]}
  comparison_dict['alexnet_singlelayer_correct'] = {'mistaken': [], 'non_mistaken':[]}
  comparison_dict[model] = {'mistaken': [], 'non_mistaken':[]}

  comparison_dict['all_correct']['mistaken'] = set(prediction_dict['alexnet_singlelayer']['correct_mistaken']).intersection(prediction_dict[model]['correct_mistaken'])
  comparison_dict['all_correct']['non_mistaken'] = set(prediction_dict['alexnet_singlelayer']['correct_non_mistaken']).intersection(prediction_dict[model]['correct_non_mistaken'])

  comparison_dict['all_wrong']['mistaken'] = set(prediction_dict['alexnet_singlelayer']['wrong_mistaken']).intersection(prediction_dict[model]['wrong_mistaken'])
  comparison_dict['all_wrong']['non_mistaken'] = set(prediction_dict['alexnet_singlelayer']['wrong_non_mistaken']).intersection(prediction_dict[model]['wrong_non_mistaken'])

  comparison_dict['alexnet_singlelayer_correct']['mistaken'] = set(prediction_dict['alexnet_singlelayer']['correct_mistaken'])- set(prediction_dict[model]['correct_mistaken'])
  comparison_dict['alexnet_singlelayer_correct']['non_mistaken'] = set(prediction_dict['alexnet_singlelayer']['correct_non_mistaken'])- set(prediction_dict[model]['correct_non_mistaken'])

  comparison_dict[model]['mistaken'] = set(prediction_dict[model]['correct_mistaken'])- set(prediction_dict['alexnet_singlelayer']['correct_mistaken'])
  comparison_dict[model]['non_mistaken'] = set(prediction_dict[model]['correct_non_mistaken'])- set(prediction_dict['alexnet_singlelayer']['correct_non_mistaken'])
  
  # print result
  print("Alexnet_singlelayer vs ", model)
  
  for (term, value) in comparison_dict.items():
    if term in list_model and term not in ['alexnet_singlelayer', model]:
      continue
    print(term, ": ")
  
    for (item, v) in value.items():
      print(item, ": ", len(v))




('Alexnet_singlelayer vs ', 'alexnet_4_1')
('alexnet_4_1', ': ')
('non_mistaken', ': ', 141)
('mistaken', ': ', 15)
('all_wrong', ': ')
('non_mistaken', ': ', 161)
('mistaken', ': ', 83)
('all_correct', ': ')
('non_mistaken', ': ', 517)
('mistaken', ': ', 154)
('alexnet_singlelayer_correct', ': ')
('non_mistaken', ': ', 40)
('mistaken', ': ', 39)
('Alexnet_singlelayer vs ', 'alexnet_7_1')
('alexnet_7_1', ': ')
('non_mistaken', ': ', 87)
('mistaken', ': ', 37)
('alexnet_singlelayer_correct', ': ')
('non_mistaken', ': ', 90)
('mistaken', ': ', 19)
('all_wrong', ': ')
('non_mistaken', ': ', 215)
('mistaken', ': ', 61)
('all_correct', ': ')
('non_mistaken', ': ', 467)
('mistaken', ': ', 174)
('Alexnet_singlelayer vs ', 'alexnet_11_1')
('alexnet_singlelayer_correct', ': ')
('non_mistaken', ': ', 25)
('mistaken', ': ', 46)
('alexnet_11_1', ': ')
('non_mistaken', ': ', 159)
('mistaken', ': ', 7)
('all_wrong', ': ')
('non_mistaken', ': ', 143)
('mistaken', ': ', 91)
('all_correct', ': ')
('non

In [9]:
 comparison_dict['alexnet_4_1']['mistaken']

{u'31QNSG6A5RSC86TKB0AQA3MDLPD87S_5_Doll03',
 u'31QNSG6A5RSC86TKB0AQA3MDLPD87S_6_Doll06',
 u'31T4R4OBOSFEK9JXSCQVCPIWF23C7J_4_Doll15',
 u'31T4R4OBOSFEK9JXSCQVCPIWF23C7J_5_Doll15',
 u'34Q075JO1XCGFV9FRGG2GEPI2G210R_4_Doll15',
 u'3B4YI393V9VGAOSLD1E6MLIANT1SSY_2_Doll01',
 u'3I7DHKZYGNZ60HPOVL1ZMBP3KKI5FB_5_Doll18',
 u'3IRIK4HM3AJVG95S92L36BI0HEZ6C1_5_Doll15',
 u'3MMN5BL1WZ39SCU13PHJ24BS3N2M34_3_Doll11',
 u'3NC5L260MOLS8RV600XYVHPU1MSFO9_6_Doll04',
 u'3OLF68YTN9036N15QHSGUM5PC30FAI_5_Doll16',
 u'3X4JMASXCM8HSTR7TD9RMQ2RLPZ0B6_6_Doll04',
 u'3X4JMASXCM8HSTR7TD9RMQ2RQWS0BI_0_Doll17',
 u'3X4JMASXCM8HSTR7TD9RMQ2RQWS0BI_4_Doll17',
 u'3ZAK8W07I4DWOS0DT4QMRGUFDO10UD_4_Doll15'}

Get prediction of multiple model in a scene:

In [36]:
# read file concate_indices:

scene_id = '3CN4LGXD5XNU4P2JM6AD64Q5QMOY4H'

model_prediction = {}
for (scene_frame_character, index) in concat_indices.items():

  # check if index is for test set
  if split_mask[index] != 'test':
    continue
    
  if scene_id not in scene_frame_character:
    continue
  
  model_prediction[scene_frame_character] = {}
  model_prediction[scene_frame_character]['correct'] = y_dict['correct_label'][index]
  for model in list_model:
    model_prediction[scene_frame_character][model] = (int(y_dict[model]['y_hat'][index])== y_dict['correct_label'][index], round(float(y_dict[model]['y_prob'][index]), 3))
      

for (scene_frame_character, value) in model_prediction.items():
  print(scene_frame_character)
  print(value)


3CN4LGXD5XNU4P2JM6AD64Q5QMOY4H_6_Doll18
{'alexnet_15_1': (True, 0.425), 'alexnet_11_1': (True, 0.437), 'alexnet_singlelayer': (False, 0.951), 'alexnet_7_1___2': (True, 0.443), 'alexnet_4_1': (True, 0.439), 'alexnet_7_3_1': (True, 0.384), 'alexnet_14_7_1': (True, 0.49), 'correct': 0}
3CN4LGXD5XNU4P2JM6AD64Q5QMOY4H_2_Doll06
{'alexnet_15_1': (True, 0.718), 'alexnet_11_1': (True, 0.73), 'alexnet_singlelayer': (True, 1.0), 'alexnet_7_1___2': (True, 0.743), 'alexnet_4_1': (True, 0.73), 'alexnet_7_3_1': (True, 0.544), 'alexnet_14_7_1': (True, 0.528), 'correct': 1}
3CN4LGXD5XNU4P2JM6AD64Q5QMOY4H_5_Doll17
{'alexnet_15_1': (True, 0.723), 'alexnet_11_1': (True, 0.726), 'alexnet_singlelayer': (True, 1.0), 'alexnet_7_1___2': (True, 0.746), 'alexnet_4_1': (True, 0.764), 'alexnet_7_3_1': (True, 0.543), 'alexnet_14_7_1': (True, 0.522), 'correct': 1}
3CN4LGXD5XNU4P2JM6AD64Q5QMOY4H_0_Doll18
{'alexnet_15_1': (True, 0.604), 'alexnet_11_1': (True, 0.61), 'alexnet_singlelayer': (False, 0.303), 'alexnet_7_1_

**Extract scene description from scene json file**

In [0]:
# load scene file
# load json file
with open("data/scenes.json") as file:
    scenes = json.load(file)

In [0]:
scene_dict_description = {}
for scene in scenes:
  scene_dict_description[scene['assignmentId']] = scene['hitDescription']

In [74]:
)scene_dict_description)

1213

In [78]:
for (key, value) in scene_dict_description.items()[:1]:
  print(key, value)

(u'3L4PIM1GQTF15V4HXI6QEX1FFNDYRC', u'{"scene_description":"The man thought he would be done barbequeing before dark.","frame_description":["A man prepares to BBQ.","The man struggles to get the flame started and gets frustrated.","The man decides to make a campfire in frustration.","The man grabs his plates of meat.","He places the meat on the flames.","The man sits down to wait for his meat to cook.","The man checks on his meat on the pit and sees it is cooking too slowly.","The man packs up his stuff and goes home in frustration."]}')
